In [3]:
# Importing needed modules
import torch
import torch.nn as nn
from tqdm import tqdm
from torch import optim
from timm import data, loss
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

In [4]:
# Train augmentation
transform_train = transforms.Compose([
    
    transforms.Resize(224),
    transforms.ColorJitter(0.3, 0.3, 0.3, 0.1),
    transforms.RandomGrayscale(0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
    transforms.RandomPerspective(0.5, 0.2),
    transforms.GaussianBlur(3, (0.1, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071,0.4867,0.4408),(0.2675,0.2565,0.2761)),
    transforms.RandomErasing(0.3, (0.05, 0.2))
    
])

In [5]:
# Test augmentation
transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5071,0.4867,0.4408),(0.2675,0.2565,0.2761))
])

In [6]:
mix = data.Mixup(
    mixup_alpha=0.8,
    cutmix_alpha=0.8,
    prob=1.0,
    switch_prob=0.5,
    mode="batch",
    label_smoothing=0.1,
    num_classes=100
)

In [7]:
# Downloading CIFAR100 dataset
train_dataset = datasets.CIFAR100(root="./data", train=True, download=False, transform=transform_train)
test_dataset = datasets.CIFAR100(root="./data", train=False, download=False, transform=transform_test)

In [8]:
# Split the training dataset into training and validation subsets.
val_ratio = 0.1
train_size = int((1 - val_ratio) * len(train_dataset))
val_size = len(train_dataset) - train_size

train_data, val_data = random_split(train_dataset, [train_size, val_size])

In [9]:
print(f"Train size: {train_size} | Test size: {len(test_dataset)} | Validation size: {val_size}")

Train size: 45000 | Test size: 10000 | Validation size: 5000


In [10]:
# DataLoaders: train (shuffled), val/test (no shuffle)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [11]:
# Train function
def train_one_epoch(model, train_loader, optimizer, loss_fn, device, scheduler=None, mix_fn=None):
    model.train()
    running_loss = 0
    total = 0
    
    for batch_idx, data in enumerate(train_loader):
        
        images, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)

        if mix_fn is not None:
            images, labels = mix_fn(images, labels)

        optimizer.zero_grad()
        
        # Forward
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        # Backward
        loss.backward()
        optimizer.step()

        if scheduler is not None:
            scheduler.step()
                    
        running_loss += loss.item() * images.size(0)
        total += labels.size(0)

    avg_loss = running_loss / total
    
    return avg_loss

In [12]:
# Evaluation function
def evaluate(model, test_loader, loss_fn, device):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            outputs = model(images)
            loss = loss_fn(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = running_loss / total
    accuracy = 100 * (correct / total)

    return avg_loss, accuracy

In [13]:
def unfreeze_layers(model, stage):

    for p in model.parameters():
        p.requires_grad = False

    if stage >= 1:
        for p in model.fc.parameters():
            p.requires_grad = True

    if stage >= 2:
        for p in model.layer4.parameters():
            p.requires_grad = True
        for p in model.layer3.parameters():
            p.requires_grad=True

    if stage >= 3:
        for p in model.parameters():
            p.requires_grad = True


    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())

    tqdm.write(f"Trainable params: {trainable_params:,} / {total_params:,}")

In [14]:
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, 100)

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet50.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
loss_fn_train = loss.SoftTargetCrossEntropy()
loss_fn_eval = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, resnet50.parameters()), lr=0.01, weight_decay=5e-4, momentum=0.9, nesterov=True)

In [17]:
best_val_loss = float("inf")
patience_counter = 1
max_patience = 10
current_stage = 1

In [16]:
for epoch in tqdm(range(100), desc="Training Epochs", ncols=100):

    if epoch == 0 and current_stage == 1:

        tqdm.write("Stage 1: Training only FC layer")
        unfreeze_layers(resnet50, current_stage)
        
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=10, pct_start=0.3, anneal_strategy="cos")
        current_stage = 2
    
    if epoch == 10 and current_stage == 2:
        
        tqdm.write("Stage 2: Unfreezing layer4 + layer3")
        unfreeze_layers(resnet50, current_stage)

        optimizer = optim.SGD([
            {'params': resnet50.layer3.parameters(), 'lr': 0.0005},
            {'params': resnet50.layer4.parameters(), 'lr': 0.001},
            {'params': resnet50.fc.parameters(), 'lr': 0.005}
        ], momentum=0.9, weight_decay=5e-4, nesterov=True)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=[0.0005, 0.001, 0.005], steps_per_epoch=len(train_loader), epochs=15, pct_start=0.3, anneal_strategy="cos")
        patience_counter = 0
        current_stage = 3
        
    if epoch == 25 and current_stage == 3:

        tqdm.write("Stage 3: Unfreezing all layers")
        unfreeze_layers(resnet50, current_stage)

        optimizer = optim.SGD([
            {'params': resnet50.layer1.parameters(), 'lr': 0.00005},
            {'params': resnet50.layer2.parameters(), 'lr': 0.0001},
            {'params': resnet50.layer3.parameters(), 'lr': 0.0003},
            {'params': resnet50.layer4.parameters(), 'lr': 0.0008},
            {'params': resnet50.fc.parameters(), 'lr': 0.002}
        ], momentum=0.9, weight_decay=5e-4, nesterov=True)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=[0.00005, 0.0001, 0.0003, 0.0008, 0.002], steps_per_epoch=len(train_loader), epochs=75, pct_start=0.3, anneal_strategy="cos")
        patience_counter = 0
        
    train_loss = train_one_epoch(resnet50, train_loader, optimizer, loss_fn_train, device, scheduler, mix)
    val_loss, val_acc = evaluate(resnet50, val_loader, loss_fn_eval, device)

    tqdm.write(f"Epoch: [{epoch + 1:03d}/100] | "
    f"Train Loss: {train_loss:.4f} | "
    f"Val Loss: {val_loss:.4f} | Acc: {val_acc:6.2f}%", end=" ")

    lrs = [f"{pg['lr']:.6f}" for pg in optimizer.param_groups]
    tqdm.write(f"| Current LR(s): [{', '.join(lrs)}]")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": resnet50.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "scheduler_state_dict": scheduler.state_dict(),
                "val_loss": val_loss,
                "val_acc": val_acc,
                "train_loss": train_loss,
                "total_params": sum(p.numel() for p in resnet50.parameters())
            },
            f"model_checkpoint.pth"
        )

        tqdm.write(f"Best model saved at epoch [{epoch + 1:03d}/100] | Val Loss: {val_loss:.5f}")
        
    else:
        patience_counter += 1
        tqdm.write(f"No improvement for {patience_counter} epochs.")
        if patience_counter >= max_patience:
            tqdm.write(f"Early stopping triggered in epoch [{epoch + 1:03d}/100] | Best Val Loss: [{best_val_loss:.6f}]")
            break

Training Epochs:   0%|                                                      | 0/100 [00:00<?, ?it/s]

Stage 1: Training only FC layer
Trainable params: 204,900 / 23,712,932


Training Epochs:   0%|                                                      | 0/100 [09:13<?, ?it/s]

Epoch: [001/100] | Train Loss: 4.4893 | Val Loss: 4.1869 | Acc:  19.72% | Current LR(s): [0.002801]


Training Epochs:   1%|▍                                         | 1/100 [09:41<15:58:57, 581.19s/it]

...

Epoch: [064/100] | Train Loss: 2.2875 | Val Loss: 1.0269 | Acc:  79.38% | Current LR(s): [0.000039, 0.000078, 0.000233, 0.000620, 0.001551]
No improvement for 8 epochs.


Training Epochs:  65%|█████████████████████████▎             | 65/100 [15:12:38<9:31:55, 980.45s/it]

Epoch: [065/100] | Train Loss: 2.3004 | Val Loss: 1.0223 | Acc:  79.34% | Current LR(s): [0.000037, 0.000075, 0.000225, 0.000600, 0.001500]
No improvement for 9 epochs.


Training Epochs:  65%|█████████████████████████▎             | 65/100 [15:33:08<8:22:27, 861.36s/it]


Epoch: [066/100] | Train Loss: 2.2987 | Val Loss: 1.0014 | Acc:  79.66% | Current LR(s): [0.000036, 0.000072, 0.000217, 0.000579, 0.001447]
No improvement for 10 epochs.
Early stopping triggered in epoch [066/100] | Best Val Loss: [0.972897]


In [22]:
# Final Evaluation
print("Loading best model...")

final_model = models.resnet50(weights="IMAGENET1K_V2")
num_features = final_model.fc.in_features
final_model.fc = nn.Linear(num_features, 100)

checkpoint = torch.load("model_checkpoint.pth")

final_model.load_state_dict(checkpoint["model_state_dict"])
final_model.to(device)

test_loss, test_acc = evaluate(final_model, test_loader, loss_fn_eval, device)
print(f"Test loss: {test_loss:.6f} | Test acc: {test_acc:.2f}%")

Loading best model...
Test loss: 0.762229 | Test acc: 84.35%
